IN THIS CODE, there was a selection on A's dets to make open shells C = singlet ms0 and D = triplet ms0
SAME goes for B to make open shells ms0 singlet and triplet
when Sa x Sb -> 3x3
when Sa x Tb -> 9x9
when Ta x Sb -> 3x3
when Ta x Tb -> 9x9

In [2]:
# last updated : 03-11-2021 Pablo

from __future__ import print_function

from scipy.linalg import eigh
import numpy

from IPython.display import display

import pandas as pd

import os

from change_all import *

THIS PART DEFINES ALL THE MATRIXES
nomenclature: Mat_a(molecule A)*******(Singlet or Triplet)_b(molecule B)*(G:ground, E:excited, O:open)s(state or shell)*******(Singlet or Triplet)


In [10]:
# last updated : 22-02-2022 Pablo
# We use all of the integrals


# Matrix of SINGLET
# g is g
# u is u
# j is g'
# v is u'


def integrals(S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):
    # Prepare all integrals
    g_h_g = 1/(2*(1+S)) * (2*eps + 2*t)
    u_h_u = 1/(2*(1-S)) * (2*eps - 2*t)
    
    g_h_j = 1/(2*(1+S)) * (2*t_i + 2*t_i2)
    u_h_v = 1/(2*(1-S)) * (2*t_i - 2*t_i2)
    
    S1 = 1/(2*(1+S)) * (2*S_i + 2*S_i2)
    S3 = 1/(2*(1-S)) * (2*S_i - 2*S_i2)
    
    gggg = 1/(4*(1+S)**2) * (2*U + 2*J + 8*lambda1 + 4*K)
    ggjj = 1/(4*(1+S)**2) * (2*Jbis + 2*J1 + 8*lambda2 + 4*omega1)
    gjjg = 1/(4*(1+S)**2) * (2*Kbis + 2*theta1 + 8*theta2 + 2*omega2 + 2*omega3)
    uuuu = 1/(4*(1-S)**2) * (2*U + 2*J - 8*lambda1 + 4*K)
    gguu = 1/(4*(1-S**2)) * (2*U + 2*J - 4*K)
    guug = 1/(4*(1-S**2)) * (2*U - 2*J)
    ggvv = 1/(4*(1-S**2)) * (2*Jbis + 2*J1 - 4*omega1)
    gvvg = 1/(4*(1-S**2)) * (2*Kbis - 2*theta1 + 2*omega2 - 2*omega3)
    uuvv = 1/(4*(1-S)**2) * (2*Jbis + 2*J1 - 8*lambda2 + 4*omega1)
    uvvu = 1/(4*(1-S)**2) * (2*Kbis + 2*theta1 - 8*theta2 + 2*omega2 + 2*omega3)
    gujv = 1/(4*(1-S**2)) * (2*Jbis - 2*J1)
    gvju = 1/(4*(1-S**2)) * (2*Kbis - 2*theta1 - 2*omega2 + 2*omega3)
    gjuv = 1/(4*(1-S**2)) * (2*Kbis + 2*theta1 - 2*omega2 - 2*omega3)
    gggj = 1/(4*(1+S)**2) * (2*gamma1 + 2*gamma2 + 2*gamma3 + 2*gamma4 + 4*gamma5 + 4*gamma6)
    gjuu = 1/(4*(1-S**2)) * (2*gamma1 + 2*gamma2 + 2*gamma3 + 2*gamma4 - 4*gamma5 - 4*gamma6)
    guuj = 1/(4*(1-S**2)) * (2*gamma1 - 2*gamma2 + 2*gamma3 - 2*gamma4)
    uuuv = 1/(4*(1-S)**2) * (2*gamma1 + 2*gamma2 - 2*gamma3 - 2*gamma4 - 4*gamma5 + 4*gamma6)
    gguv = 1/(4*(1-S**2)) * (2*gamma1 + 2*gamma2 - 2*gamma3 - 2*gamma4 + 4*gamma5 - 4*gamma6)
    guvg = 1/(4*(1-S**2)) * (2*gamma1 - 2*gamma2 - 2*gamma3 + 2*gamma4)
    
    
    return g_h_g, u_h_u, g_h_j, u_h_v, S, S1, S3, gggg, ggjj, gjjg, uuuu, gguu, guug, ggvv, gvvg, uuvv, uvvu, gujv, gvju, gjuv, gggj, gjuu, guuj, uuuv, gguv, guvg


def Mat_aSinglet_bSinglet(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):

    ### Normalize given coefficients
    norm = coeff_gs ** 2 + coeff_es ** 2 + coeff_os ** 2
    
    E1 = coeff_gs * numpy.sqrt(1 / norm)
    E2 = coeff_es * numpy.sqrt(1 / norm)
    E3 = coeff_os * numpy.sqrt(1 / norm)
    
    g_h_g, u_h_u, g_h_j, u_h_v, S, S1, S3, gggg, ggjj, gjjg, uuuu, gguu, guug, ggvv, gvvg, uuvv, uvvu, gujv, gvju, gjuv, gggj, gjuu, guuj, uuuv, gguv, guvg = integrals(S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)

    
    ### PREPARE THE MATRIXES
    ## b = GS singlet 
    
    # Diagonal elements of Singlet Matrix
    a1a1 = 4*S1**2*gjjg - 8*S1*gggj + 4*g_h_g*(1 - S1**2) + 4*g_h_j*(S1**3 - S1) + 2*(1 - S1**2)*(ggjj - gjjg) + gggg + 2*ggjj + gggg
    b1b1 = 2*g_h_g + 2*u_h_u + gggg + 4*ggvv + uuuu - 2*gvvg
    c1c1 = -2*S1*g_h_j - 2*S1*gjuu - 2*S1*gggj + g_h_g*(1 - S1**2) + 2*g_h_g + u_h_u*(1 - S1**2) + (1 - S1**2)*(ggvv - gvvg) + 2*ggjj + gguu + gggg + ggvv - gjjg
    d1d1 = c1c1
    # Extra diagonal term
    a1b1 = S1**2*gvvg - 2*S1*guuj + guug 
    a1c1 = 0
    a1d1 = 0
    b1c1 = 0
    b1d1 = 0
    c1d1 = a1b1
    
    ## b = ES singlet
        
    
    # Diagonal elements of Singlet Matrix
    a2a2 = b1b1
    b2b2 = 4*S3**2*uvvu - 8*S3*uuuv + 4*u_h_u*(1 - S3**2) + 4*u_h_v*(S3**3 - S3) + 2*(1 - S3**2)*(uuvv - uvvu) + uuuu + 2*uuvv + uuuu
    c2c2 = -2*S3*u_h_v - 2*S3*gguv - 2*S3*uuuv + g_h_g*(1 - S3**2) + u_h_u*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*(ggvv - gvvg) + gguu + ggvv + 2*uuvv + uuuu - uvvu
    d2d2 = c2c2
    # Extra diagonal term
    a2b2 = S3**2*gvvg - 2*S3*guvg + guug
    a2c2 = 0
    a2d2 = 0
    b2c2 = 0
    b2d2 = 0
    c2d2 = a2b2
    
    
    
    
    ## b = OS singlet |g -u|
    
 
    # Diagonal elements of Singlet Matrix
    a3a3 = -2*S1*g_h_j - 2*S1*gggj - 2*S1*gjuu + g_h_g*(1 - S1**2) + 2*g_h_g + u_h_u*(1 - S1**2) + (1 - S1**2)*(ggvv - gvvg) + gggg + 2*ggjj + ggvv + gguu - gjjg
    b3b3 = -2*S3*u_h_v - 2*S3*gguv - 2*S3*uuuv + g_h_g*(1 - S3**2) + u_h_u*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*(ggvv - gvvg) + gguu + ggvv + uuuu + 2*uuvv - uvvu
    c3c3 = 4*S1*S3*gjuv - 4*S1*gjuu - 4*S3*gguv + 2*g_h_g*(1 - S3**2) + 2*g_h_j*(S1*S3**2 - S1) + 2*u_h_u*(1 - S1**2) + 2*u_h_v*(S1**2*S3 - S3) + (1 - S1**2)*(uuvv - uvvu) + (1 - S3**2)*(ggjj - gjjg) + gguu + ggvv + gguu + ggvv
    d3d3 = 2*g_h_g + 2*u_h_u + ggjj + ggvv + 2*gguu + ggvv + uuvv - 2*gvvg
    
    # Extra diagonal term
    a3b3 = S1*S3*gvju - S1*guuj - S3*guvg + guug
    a3c3 = 0
    a3d3 = 0
    b3c3 = 0
    b3d3 = 0
    c3d3 = a3b3
    
    ## b = OS singlet |u -g|
    
    # Diagonal elements of Singlet Matrix
    a4a4 = a3a3
    b4b4 = b3b3
    c4c4 = d3d3
    d4d4 = c3c3
    
    # Extra diagonal term
    a4b4 = a3b3
    a4c4 = 0
    a4d4 = 0
    b4c4 = 0
    b4d4 = 0
    c4d4 = a4b4
    
    
    ## EXTRA COUPLINGS
    # The next part was computed automatically using the script (20220209_FASTsymbolic...)
    
    a1a2 = a3a4 = S1**2*gvvg - 2*S1*guuj + guug
    d1d2 = c1c2 = c3c4 = d3d4 = S1*S3*gvju - S1*guuj - S3*guvg + guug
    b1b2 = b3b4 = S3**2*gvvg - 2*S3*guvg + guug
   
    a1b2 = a1c2 = a1d2 = a1b3 = a1b4 = 0
    b1a2 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j - 2*S1*S3*(gujv - gjuv) + 2*S1*S3*gjuv + S3**2*gjjg
    b1c2 = b1d2 = b1a3 = c1d3 = 0
    c1d2 = -S1*S3*(gujv - gvju)
    c1b2 = b1a4 = c1a2 = a2b4 = a2b3 = d1c4 = d1c3 = 0
    d1c2 = c1d2
    d1b2 = d1a2 = c1d4 = c2d4 = c2d3 = b2a4 = b2a3 = d2c3 = d2c4 = 0
    a3b4 = c1d2

    a3c4 = a3d4 = 0 
    b3a4 = c1d2
    b3c4 = b3d4 = c3a4 = c3b4 = c3d4 = d3a4 = d3b4 = 0
    d3c4 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j - 2*S1*S3*gujv + 4*S1*S3*gjuv + S3**2*gjjg #4 MOs of diff ?

    a1c3 = (1-S1**2)*(gujv - gvju) 
    b2c3 = (1-S3**2)*(gujv - gvju)
    d2a3 = c2a4 = gujv - gvju
    b2d4 = b2c3
    d1b3 = c1b4 = d2a3
    a1d4 = a1c3 

    a1d3 = d1a3 = c1a4 = a1c4 = S1**2*gvju - 2*S1*guvg + gujv
    b2c4 = b2d3 = d2b3 = c2b4 = S3**2*gvju - 2*S3*guuj + gujv
    d2a4 = c2a3 = d1b4 = b1c3 = a2d4 = a2c3 =  c1b3 = b1d4 = S1*S3*gvvg - S1*guvg - S3*guuj + gujv

    b1d3 = b1c4 = -S1*S3*g_h_g - S1*S3*u_h_u - S1*S3*(ggvv - gvvg) - S1*u_h_v - S1*uuuv - S1*gguv - S3*g_h_j - S3*gggj - S3*gjuu + gujv - gjuv
    a2c4 = a2d3 = -S1*S3*g_h_g - S1*S3*u_h_u - S1*S3*(ggvv - gvvg) - S1*u_h_v - S1*gguv - S1*uuuv - S3*g_h_j - S3*gggj - S3*gjuu - gjuv + gujv
    d2b4 = c2b3 = 2*S1*S3**2*u_h_v - 2*S1*S3*u_h_u - S1*S3*(uuvv - uvvu) + 2*S1*S3*uvvu - S1*uuuv - S1*uuuv + 2*S3**2*gjuv - 2*S3*gjuu + g_h_j*(S3**3 - S3) + u_h_v*(S1*S3**2 - S1) + (1 - S3**2)*(gujv - gjuv)
    d1a4 = c1a3 = 2*S1**2*S3*g_h_j + 2*S1**2*gjuv - 2*S1*S3*g_h_g - S1*S3*(ggjj - gjjg) + 2*S1*S3*gjjg - 2*S1*gguv - 2*S3*gggj + g_h_j*(S1**2*S3 - S3) + u_h_v*(S1**3 - S1) + (1 - S1**2)*(gujv - gjuv)

    a1a3 = a1a4 = b1b3 = b1b4 = c1c3 = c1c4 = d1d3 = d1d4 = a2a3 = a2a4 = b2b3 = b2b4 = c2c3 = c2c4 = d2d3 = d2d4 = 0
  
    ### MATRIXES
    
    ## Effets nucléaires
    DIAG = 2/L + 2/numpy.sqrt(L**2 + l**2) + 2/l
    # Diagonal terms
    aa = DIAG + E1**2 * a1a1 + E2 **2 * a2a2 + 1/2 * E3 **2 * (a3a3 + a4a4 + 2*a3a4) \
            + 2*E1*E2*a1a2 + numpy.sqrt(2)*E1*E3*(a1a3 + a1a4) + numpy.sqrt(2)*E2*E3*(a2a3 + a2a4)
    
    bb = DIAG + E1**2 * b1b1 + E2 **2 * b2b2 + 1/2 * E3 **2 * (b3b3 + b4b4 + 2*b3b4) \
            + 2*E1*E2*b1b2 + numpy.sqrt(2)*E1*E3*(b1b3 + b1b4) + numpy.sqrt(2)*E2*E3*(b2b3 + b2b4)
    
    cc = DIAG + 1/2 * (E1**2 * (c1c1 + d1d1 + 2*c1d1) + E2**2 * (c2c2 + d2d2 + 2*c2d2) \
            + 1/2 * E3**2 * (c3c3 + d3d3 + 2*c3d3 + c4c4 + d4d4 + 2*c4d4 \
            + 2*(c3c4 + d3d4) + 2*(c3d4 + d3c4)) + 2*E1*E2*(c1c2 + d1d2 + c1d2 + d1c2) \
            + numpy.sqrt(2)*E1*E3*(c1c3 + c1d3 + d1c3 + d1d3 + c1c4 + c1d4 + d1c4 + d1d4) \
            + numpy.sqrt(2)*E2*E3*(c2c3 + c2d3 + d2c3 + d2d3 + c2c4 + c2d4 + d2c4 + d2d4))
    
    # Extra diagonal
    ab = E1**2 * a1b1 + E2 **2 * a2b2 + 1/2 * E3 **2 * (a3b3 + a4b4 + a3b4 + b3a4) \
            + E1*E2*(a1b2 + b1a2) + 1/numpy.sqrt(2) *E1*E3*(a1b3 + b1a3 + a1b4 + b1a4) \
            + 1/numpy.sqrt(2)*E2*E3*(a2b3 + b2a3 + a2b4 + b2a4)
    
    ac = 1/numpy.sqrt(2)*(E1**2 * (a1c1 + a1d1) + E2**2 * (a2c2 + a2d2) \
            + 1/2 * E3**2 *(a3c3 + a4c4 + a3c4 + c3a4 + a3d3 + a4d4 + a3d4 + d3a4) \
            + E1*E2 * (a1c2 + c1a2 + a1d2 + d1a2) \
            + 1/numpy.sqrt(2)*E1*E3 * (a1c3 + a1c4 + c1a3 + c1a4 + a1d3 + a1d4 + d1a3 + d1a4) \
            + 1/numpy.sqrt(2)*E2*E3 * (a2c3 + a2c4 + c2a3 + c2a4 + a2d3 + a2d4 + d2a3 + d2a4))
    
    bc = 1/numpy.sqrt(2)*(E1**2 * (b1c1 + b1d1) + E2**2 * (b2c2 + b2d2) \
            + 1/2 * E3**2 *(b3c3 + b4c4 + b3c4 + c3b4 + b3d3 + b4d4 + b3d4 + d3b4) \
            + E1*E2 * (b1c2 + c1b2 + b1d2 + d1b2) \
            + 1/numpy.sqrt(2)*E1*E3 * (b1c3 + b1c4 + c1b3 + c1b4 + b1d3 + b1d4 + d1b3 + d1b4) \
            + 1/numpy.sqrt(2)*E2*E3 * (b2c3 + b2c4 + c2b3 + c2b4 + b2d3 + b2d4 + d2b3 + d2b4))
    
    ### MATRIX FOR TRIPLET (1 term)
    dd = DIAG + 1/2 * (E1**2 * (c1c1 + d1d1 - 2*c1d1) + E2**2 * (c2c2 + d2d2 - 2*c2d2) \
            + 1/2 * E3**2 * (c3c3 + c4c4 + 2*c3c4 + d3d3 + d4d4 + 2*d3d4 \
            - 2*(c3d3 + c4d4) - 2*(c3d4 + d3c4)) + 2*E1*E2*(c1c2 + d1d2 - c1d2 - d1c2) \
            + numpy.sqrt(2)*E1*E3*(c1c3 - c1d3 - d1c3 + d1d3 + c1c4 - c1d4 - d1c4 + d1d4) \
            + numpy.sqrt(2)*E2*E3*(c2c3 - c2d3 - d2c3 + d2d3 + c2c4 - c2d4 - d2c4 + d2d4))
    
    
    
    
    
    
    # compute the matrix
    H = [[aa, ab, ac],[ab, bb, bc],[ac, bc, cc]]
    return numpy.array(H), dd



def Mat_aSinglet_bTriplet(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):

    ### Normalize given coefficients
    coeff_plus = coeff_minus = coeff_0 = 1
    norm = coeff_plus ** 2 + coeff_minus ** 2 + coeff_0 ** 2
    
    E1 = coeff_plus * numpy.sqrt(1 / norm)
    E2 = coeff_minus * numpy.sqrt(1 / norm)
    E3 = coeff_0 * numpy.sqrt(1 / norm)
    

    
    g_h_g, u_h_u, g_h_j, u_h_v, S, S1, S3, gggg, ggjj, gjjg, uuuu, gguu, guug, ggvv, gvvg, uuvv, uvvu, gujv, gvju, gjuv, gggj, gjuu, guuj, uuuv, gguv, guvg = integrals(S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)

    
    ### PREPARE THE MATRIXES
    ## b = +1
    #-2*S3*u*v + S3*(-<j**2|u*v> + <j*u|j*v>) + S3*(<j*u|j*v> - <u*v|j**2>) - S3*<g**2|u*v> - S3*<u*v|g**2> + 2*g**2*(1 - S3**2) + 2*u**2 + (1 - S3**2)*<g**2|j**2> + <g**2|v**2> + <j**2|v**2> + <u**2|g**2> + <u**2|j**2> + <u**2|v**2> - <j*u|j*u> - <j*v|j*v> - <u*v|u*v>
    # Diagonal elements of Singlet Matrix
    a1a1 = -2*S1*g_h_j - 2*S1*(gjuu - guuj) - 2*S1*gggj + g_h_g*(1 - S1**2) + 2*g_h_g + u_h_u*(1 - S1**2) + (1 - S1**2)*ggvv + gggg + 2*ggjj + ggvv + gguu - gjjg - gvvg - guug
    b1b1 = -2*S3*u_h_v + 2*S3*(guvg - gguv) - 2*S3*uuuv + g_h_g*(1 - S3**2) + u_h_u*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*ggvv + gguu + ggvv + uuuu + 2*uuvv - gvvg - guug - uvvu
    c1c1 = -2*S1*g_h_j - 2*S1*(gjuu - guuj) - 2*S1*gjuu + 2*g_h_g + 2*u_h_u*(1 - S1**2) + (1 - S1**2)*uuvv + ggjj + gguu + ggvv + gguu + ggvv - gjjg - gvvg - guug
    d1d1 = -2*S3*u_h_v - 2*S3*(gguv - guvg) - 2*S3*gguv + 2*u_h_u + 2*g_h_g*(1 - S3**2) + (1 - S3**2)*ggjj + ggvv + gguu + gguu + ggvv + uuvv - gvvg - guug - uvvu
    # Extra diagonal term
    a1b1 = S1*S3*gujv - S1*guuj - S3*guvg + guug
    a1c1 = 0
    a1d1 = 0
    b1c1 = 0
    b1d1 = 0
    c1d1 = a1b1
    

    ## b = -1
    
    # Diagonal elements of Singlet Matrix
    a2a2 = a1a1
    b2b2 = b1b1
    c2c2 = d1d1
    d2d2 = c1c1
    # Extra diagonal term
    a2b2 = a1b1
    a2c2 = a1d1
    a2d2 = a1c1
    b2c2 = b1d1
    b2d2 = b1c1
    c2d2 = a2b2
    
    ## b = OS singlet |g -u|
    
 
    # Diagonal elements of Singlet Matrix
    a3a3 = -2*S1*g_h_j - 2*S1*gggj - 2*S1*gjuu + g_h_g*(1 - S1**2) + 2*g_h_g + u_h_u*(1 - S1**2) + (1 - S1**2)*(ggvv - gvvg) + gggg + 2*ggjj + ggvv + gguu - gjjg
    b3b3 = -2*S3*u_h_v - 2*S3*gguv - 2*S3*uuuv + g_h_g*(1 - S3**2) + u_h_u*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*(ggvv - gvvg) + gguu + ggvv + uuuu + 2*uuvv - uvvu
    c3c3 = 4*S1*S3*gjuv - 4*S1*gjuu - 4*S3*gguv + 2*g_h_g*(1 - S3**2) + 2*g_h_j*(S1*S3**2 - S1) + 2*u_h_u*(1 - S1**2) + 2*u_h_v*(S1**2*S3 - S3) + (1 - S1**2)*(uuvv - uvvu) + (1 - S3**2)*(ggjj - gjjg) + gguu + ggvv + gguu + ggvv
    d3d3 = 2*g_h_g + 2*u_h_u + ggjj + ggvv + 2*gguu + ggvv + uuvv - 2*gvvg
    
    # Extra diagonal term
    a3b3 = S1*S3*gvju - S1*guuj - S3*guvg + guug
    a3c3 = 0
    a3d3 = 0
    b3c3 = 0
    b3d3 = 0
    c3d3 = a3b3
    
    ## b = OS singlet |u -g|
    
    # Diagonal elements of Singlet Matrix
    a4a4 = a3a3
    b4b4 = b3b3
    c4c4 = d3d3
    d4d4 = c3c3
    
    # Extra diagonal term
    a4b4 = a3b3
    a4c4 = 0
    a4d4 = 0
    b4c4 = 0
    b4d4 = 0
    c4d4 = a4b4
    
    
    ## EXTRA COUPLINGS=
    # The next part was computed automatically using the script (20220209_FASTsymbolic...)

    a1a2 = d1d2 = c1c2 = b1b2 = 0

    a3a4 = S1**2*gvvg - 2*S1*guuj + guug
    b3b4 = S3**2*gvvg - 2*S3*guvg + guug
    c3c4 = d3d4 = S1*S3*gvju - S1*guuj - S3*guvg + guug

    a1b2 = a1c2 = a1d2 = a1b3 = a1b4 = 0
    b1a2 = 0
    b1c2 = b1d2 = b1a3 = c1d3 = 0
    c1d2 = 0
    c1b2 = b1a4 = c1a2 = a2b4 = a2b3 = d1c4 = d1c3 = 0
    d1c2 = 0
    d1b2 = d1a2 = c1d4 = c2d4 = c2d3 = b2a4 = b2a3 = d2c3 = d2c4 = 0

    a3b4 = -S1*S3*(gujv - gvju)
    a3c4 = a3d4 = 0 
    b3a4 = a3b4
    b3c4 = b3d4 = c3a4 = c3b4 = c3d4 = d3a4 = d3b4 = 0
    d3c4 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j - 2*S1*S3*gujv + 4*S1*S3*gjuv + S3**2*gjjg #4 MOs of diff ?
    
    a1c3 = 0
    b2c3 = 0
    d2a3 = c2a4 = 0
    b2d4 = 0
    d1b3 = c1b4 = 0
    a1d4 = 0 
    
    a1d3 = b2c4 = b2d3 = d2a4 = c2a3 = d1b4 = a1c4 = b1c3 = a2d4 = d2b3 = \
           c2b4 = a2c3 = d1a3 = c1a4 = c1b3 = b1d4 = 0 

    b1d3 = a2c4 = a2d3 = 0
    d2b4 = c2b3 = 0
    d1a4 = c1a3 = 0
    b1c4 = 0
    
    a1a3 = a1a4 = 0
    b1b3 = b1b4 = 0
    c1c3 = 0
    c1c4 = d1d3 = 0
    d1d4 = 0
    a2a3 = a2a4 = 0
    b2b3 = b2b4 = 0
    c2c3 = 0
    c2c4 = d2d3 = 0
    d2d4 = 0
    
   # Terms for ei and fi
    e1e1 = 2*S1*S3*(gjuv - gujv) + 2*S1*S3*(gjuv - gvju) + 4*S1*(guuj - gjuu) - 4*S3*(gguv - guvg) + 2*g_h_g*(1 - S3**2) + 2*g_h_j*(S1*S3**2 - S1) + 2*u_h_u*(1 - S1**2) + 2*u_h_v*(S1**2*S3 - S3) + (1 - S1**2)*(uuvv - uvvu) + (1 - S3**2)*(ggjj - gjjg) + 2*gguu + 2*ggvv - 2*guug - 2*gvvg  
    f1f1 = 2*g_h_g + 2*u_h_u + ggjj + 2*gguu + 2*ggvv + uuvv - 2*guug
    e2e2 = f1f1
    f2f2 = e1e1

    e3e3 = -2*S1*g_h_j + 2*S1*(guuj - gjuu) - 2*S1*gjuu + 2*g_h_g + 2*u_h_u*(1 - S1**2) + (1 - S1**2)*uuvv + ggjj + 2*gguu + 2*ggvv - gjjg - guug - gvvg
    f3f3 = -2*S3*u_h_v - 2*S3*(gguv - guvg) - 2*S3*gguv + 2*g_h_g*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*ggjj + 2*gguu + 2*ggvv + uuvv - guug - gvvg - uvvu
    e4e4 = f3f3
    f4f4 = e3e3


    e1f1 = e1e2 = e1f2 = e1e3 = e1f3 = e1e4 = e1f4 = 0

    f1e2 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j + 2*S1*S3*(2*gjuv - gvju) + S3**2*gjjg

    f1f2 = f1e3 = f1f3 = f1e4 = f1f4 = e2f2 = e2e3 = e2f3 = 0
    e2e4 = e2f4 = f2e3 = f2f3 = f2e4 = f2f4 = e3f3 = e3f4 = 0

    e3e4 = f3f4 = S1*S3*gujv - S1*guuj - S3*guvg + guug
    
    f3e4 = e4f4 = 0
    
   #All the rest:


    e1a1 = e1b1 = e1c1 = e1d1 = e1a2 = e1b2 = e1c2 = e1d2 = e1a3 = e1b3 = e1c3 = e1d3 = e1a4 = e1b4 = e1c4 = e1d4 = 0
    f1a1 = f1b1 = f1c1 = f1d1 = f1a2 = f1b2 = f1c2 = f1d2 = f1a3 = f1b3 = 0 
    
    f1c3 = -S1*S3*gujv + S1*guuj + S3*guvg - gvvg
    f1d3 = 2*S3**2*g_h_g + S3**2*ggjj + 2*S3*u_h_v + S3*(gguv - guvg) - S3*(guvg - 3*gguv) + uvvu
    
    f1a4 = f1b4 = 0 
    
    f1c4 = 2*S1**2*u_h_u + S1**2*uuvv + 2*S1*g_h_j - S1*(guuj - gjuu) + S1*(gjuu - guuj) + S1*gjuu + S1*gjuu + gjjg
    f1d4 = f1c3


    e2a1 = e2b1 = e2c1 = e2d1 = e2a2 = e2b2 = e2c2 = e2d2 = e2a3 = e2b3 = 0 
    
    e2c3 = f1c3 
    e2d3 = f1c4 
    
    e2a4 = e2b4 = 0
    
    e2c4 = f1d3
    e2d4 = f1c3 

    f2a1 = f2b1 = f2c1 = f2d1 = f2a2 = f2b2 = f2c2 = f2d2 = f2a3 = f2b3 = f2c3 = f2d3 = f2a4 = f2b4 = f2c4 = f2d4 = e3a1 = e3b1 = 0 
    
    e3c1 = 2*S1*S3**2*g_h_j + S1*S3*(2*gjuv - gvju) - S1*S3*(gujv - 2*gjuv) - 2*S3**2*g_h_g - S3**2*(ggjj - gjjg) - 2*S3*(gguv - guvg) - 2*S3*gguv + 2*u_h_v*(S1**2*S3 - S3) - (1 - S1**2)*uvvu
    e3d1 = S1*S3*gvju - S1*guuj - S3*guvg + gvvg

    e3a2 = e3b2 = e3c2 = e3d2 = e3a3 = e3b3 = e3c3 = e3d3 = e3a4 = e3b4 = e3c4 = e3d4 = f3a1 = f3b1 = f3c1 = f3d1 = f3a2 = f3b2 = 0 
    
    f3c2 = 2*S1**2*S3*u_h_v - 2*S1**2*u_h_u - S1**2*(uuvv - uvvu) - S1*S3*(gujv - gjuv) + S1*S3*(gjuv - gvju) + 2*S1*S3*gjuv + S1*(guuj - gjuu) - S1*(gjuu - guuj) - 2*S1*gjuu + 2*g_h_j*(S1*S3**2 - S1) - (1 - S3**2)*gjjg
    f3d2 = e3d1

    f3a3 = f3b3 = f3c3 = f3d3 = f3a4 = f3b4 = f3c4 = f3d4 = e4a1 = e4b1 = 0 
    
    e4c1 = e3d1
    e4d1 = f3c2
    
    e4a2 = e4b2 = e4c2 = e4d2 = e4a3 = e4b3 = e4c3 = e4d3 = e4a4 = e4b4 = e4c4 = e4d4 = f4a1 = f4b1 = f4c1 = f4d1 = f4a2 = f4b2 = 0
    
    f4c2 = e3d1
    f4d2 = e3c1
    
    f4a3 = f4b3 = f4c3 = f4d3 = f4a4 = f4b4 = f4c4 = f4d4 = 0
  
    ### MATRIXES
    
    ## Effets nucléaires
    DIAG = 2/L + 2/numpy.sqrt(L**2 + l**2) + 2/l
    # Diagonal terms
    aa = DIAG + a1a1 #+ E1**2 * a1a1 + E2 **2 * a2a2 + 1/2 * E3 **2 * (a3a3 + a4a4 - 2*a3a4) \
    #       + 2*E1*E2*a1a2 + numpy.sqrt(2)*E1*E3*(a1a3 - a1a4) + numpy.sqrt(2)*E2*E3*(a2a3 - a2a4)
    
    bb = DIAG + b1b1 #+ E1**2 * b1b1 + E2 **2 * b2b2 + 1/2 * E3 **2 * (b3b3 + b4b4 - 2*b3b4) \
    #       + 2*E1*E2*b1b2 + numpy.sqrt(2)*E1*E3*(b1b3 - b1b4) + numpy.sqrt(2)*E2*E3*(b2b3 - b2b4)
    
    cc = DIAG + 1/2 * (c1c1 + d1d1 + 2*c1d1) 
    cbiscbis = DIAG + 1/4 *(c3c3 + d3d3 + 2*c3d3 + c4c4 + d4d4 + 2*c4d4 - 2*(c3c4 + d3d4) - 2*(c3d4 + d3c4))
    #       + 1/2 * (E1**2 * (c1c1 + d1d1 + 2*c1d1) + E2**2 * (c2c2 + d2d2 + 2*c2d2) \
    #       + 1/2 * E3**2 * (c3c3 + d3d3 + 2*c3d3 + c4c4 + d4d4 + 2*c4d4 \
    #       - 2*(c3c4 + d3d4) - 2*(c3d4 + d3c4)) + 2*E1*E2*(c1c2 + d1d2 + c1d2 + d1c2) \
    #       + numpy.sqrt(2)*E1*E3*(c1c3 + c1d3 + d1c3 + d1d3 - c1c4 - c1d4 - d1c4 - d1d4) \
    #       + numpy.sqrt(2)*E2*E3*(c2c3 + c2d3 + d2c3 + d2d3 - c2c4 - c2d4 - d2c4 - d2d4))
    
    # Extra diagonal
    ab = a1b1 
    #       E1**2 * a1b1 + E2 **2 * a2b2 + 1/2 * E3 **2 * (a3b3 + a4b4 - a3b4 - b3a4) \
    #       + E1*E2*(a1b2 + b1a2) + 1/numpy.sqrt(2) *E1*E3*(a1b3 + b1a3 - a1b4 - b1a4) \
    #       + 1/numpy.sqrt(2)*E2*E3*(a2b3 + b2a3 - a2b4 - b2a4)
    
    ac = 1/numpy.sqrt(2)*(a1c1 + a1d1)
    #1/numpy.sqrt(2)*(E1**2 * (a1c1 + a1d1) + E2**2 * (a2c2 + a2d2) \
    #       + 1/2 * E3**2 *(a3c3 + a4c4 - a3c4 - c3a4 + a3d3 + a4d4 - a3d4 - d3a4) \
    #       + E1*E2 * (a1c2 + c1a2 + a1d2 + d1a2) \
    #       + 1/numpy.sqrt(2)*E1*E3 * (a1c3 - a1c4 + c1a3 - c1a4 + a1d3 - a1d4 + d1a3 - d1a4) \
    #       + 1/numpy.sqrt(2)*E2*E3 * (a2c3 - a2c4 + c2a3 - c2a4 + a2d3 - a2d4 + d2a3 - d2a4))
    
    bc =  1/numpy.sqrt(2)*(b1c1 + b1d1)
    #1/numpy.sqrt(2)*(E1**2 * (b1c1 + b1d1) + E2**2 * (b2c2 + b2d2) \
    #         + 1/2 * E3**2 *(b3c3 + b4c4 - b3c4 - c3b4 + b3d3 + b4d4 - b3d4 - d3b4) \
    #         + E1*E2 * (b1c2 + c1b2 + b1d2 + d1b2) \
    #         + 1/numpy.sqrt(2)*E1*E3 * (b1c3 - b1c4 + c1b3 - c1b4 + b1d3 - b1d4 + d1b3 - d1b4) \
    #         + 1/numpy.sqrt(2)*E2*E3 * (b2c3 - b2c4 + c2b3 - c2b4 + b2d3 - b2d4 + d2b3 - d2b4))
    
    ### MATRIX FOR (TT) coupling (3 term: 1S, 1T, 1Q)
    singsing = DIAG + 1/3 * (e2e2 + f1f1 + 1/4*(c3c3 + c4c4 + d3d3 + d4d4) + 2*f1e2 \
             + (e2c3 - e2c4 - e2d3 + e2d4 + f1c3 - f1c4 - f1d3 + f1d4) \
             + 2*(-c3c4 - c3d3 + c3d4 + d3c4 - c4d4 - d3d4))
    
#     triplet = DIAG + 1/2 * (e1**2 * (c1c1 + d1d1 - 2*c1d1) + e2**2 * (c2c2 + d2d2 - 2*c2d2) \
#             + 1/2 * e3**2 * (c3c3 + c4c4 - 2*c3c4 + d3d3 + d4d4 - 2*d3d4 \
#             - 2*(c3d3 + c4d4) + 2*(c3d4 + d3c4)) + 2*e1*e2*(c1c2 + d1d2 - c1d2 - d1c2) \
#             + numpy.sqrt(2)*e1*e3*(c1c3 - c1d3 - d1c3 + d1d3 - c1c4 + c1d4 + d1c4 - d1d4) \
#             + numpy.sqrt(2)*e2*e3*(c2c3 - c2d3 - d2c3 + d2d3 - c2c4 + c2d4 + d2c4 - d2d4))
    
    triptrip = DIAG + 1/2 * (e2e2 + f1f1 - 2*f1e2)
    
    quintquint = DIAG + 1/6 * (c3c3 + c4c4 + d3d3 + d4d4 + f1f1 + e2e2 + 2*(-c3c4 - c3d3 + c3d4 + e2c3 + f1c3 \
                + d3c4 - c4d4 - e2c4 - f1c4 - d3d4 - e2d3 - f1d3 + e2d4 + f1d4 + f1e2))
    
    ## EXTRA DIAG FOR triplet with a, b and c
    atrip = 1/2 * (e2a3 - e2a4 - f1a3 + f1a4)
    btrip = 1/2 * (e2b3 - e2b4 - f1b3 + f1b4) 
    ctrip = 1/(2*numpy.sqrt(2)) * (e2c3 + e2d3 - e2c4 - e2d4 - f1c3 - f1d3 + f1c4 + f1d4)
    
    
    
    #Hbis = [[c3c3,c3d3,c3c4,c3d4,e2c3,f1c3],[c3d3,d3d3,d3c4,d3d4, e2d3,f1d3], [c3c4,d3c4,c4c4,c4d4,e2c4,f1c4],[c3d4,d3d4,c4d4,d4d4,e2d4,f1d4],[e2c3,e2d3,e2c4,e2d4,e2e2,f1e2],[f1c3,f1d3,f1c4,f1d4,f1e2,f1f1]]
    
    tripms1 = DIAG + 1/4 * (e3e3 -2*e3e4 - 2*e3c1 + 2*e3d1 + e4e4 + 2*e4c1 - 2*e4d1 + c1c1 - 2*c1d1 + d1d1)
    ctripms1 = 1/(2*numpy.sqrt(2)) * (e3c1 + e3d1 - e4c1 - e4d1 - c1c1 + d1d1)

    
    
    # compute the matrix
    H = [[aa, ab, ac, atrip],[ab, bb, bc, btrip],[ac, bc, cc, ctrip],[atrip, btrip, ctrip, triptrip]]
    return numpy.array(H), singsing, quintquint#, tripms1, ctripms1, cbiscbis



This part diagonalize matrixes and computes eigenvalues/eigenvectors

In [11]:
# last updated: 03.02.2022


def Matrix_diag_b_SINGLET(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):
    """
    Scheme :
        b|     |b'
         |     |
        a|     |a'
    
    This function diagonalizes one 3x3 matrix (4e- in 4MOs):    
        - H: molecule A is SINGLET and molecule B is a linear combination of SINGLET states

    Linear combination (normalized):
        B_state = coeff_gs * B_groundstate + coeff_es * B_excitedstate + coeff_os * B_openshell
    
    """
    # Initialization of results
    result = []
    
    # Columns for the DataFrame
    col = ("E(S)1","E(S)2", "E(S)3")
    
    # DEFINE THE TWO MATRIXES    
    H, triplet_eigenvalue = Mat_aSinglet_bSinglet(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)
        
    # SCIPY METHOD of DIAGONALIZATION
    # https://coulomb.umontpellier.fr/perso/daniele.coslovich/python_for_physicists/#sec-3-3
    
    # Obtain the eigenvalues and eigenvectors of H
    eigenvalues, eigenvectors = eigh(H)
    # Extract eigenvalues as real quantities
    eigenvalues = numpy.real(eigenvalues)
    #print(eigenvectors)
        #Data treatment
    new_eigenvectors =[[],[],[]]
    new_eigenvalues = [0,0,0]
    for i in range(3):
        if eigenvectors[i][0]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[0] = eigenvectors[i]
            new_eigenvalues[0] = eigenvalues[i]
            #print("vec1", eigenvectors[i])
            
        if eigenvectors[i][1]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[1] = eigenvectors[i]
            new_eigenvalues[1] = eigenvalues[i]
            
        if eigenvectors[i][2]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[2] = eigenvectors[i]
            new_eigenvalues[2] = eigenvalues[i]
            #print("vec3", eigenvectors[i])
            
        
    result = [[val for val in new_eigenvalues]]
    df = pd.DataFrame(result, columns=col) 
    
    df["E(T)"] = triplet_eigenvalue

    return H, df, new_eigenvectors

#from change_distance import *

# Import the integrals
L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = dist3_5A_bond0_740A()

#S_norm = 0.6598731215439126
#S = S_norm
#S= 0
#S_i = S_i2 = 0
# Select which combinations of c1*phi1 (GROUND) + c2*phi2 (EXCITED) + c3*phi3 (OPEN-SHELL)
coeff_gs = 1
coeff_os = 0
coeff_es = 0

H, result_matrix, eigenvectors = Matrix_diag_b_SINGLET(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)

print("\033[4mEigenvalues for A Singlet (S) and A Triplet (T)\033[0m")
display(result_matrix)
print("\n\033[4mEigenvectors of A Singlet\033[0m")
print(pd.DataFrame(eigenvectors).to_string(index=False, header=False))

print("\n\033[4mMatrix of A Singlet\033[0m")
print(pd.DataFrame(H).to_string(index=False, header=False))


#display(eigenvectorsbis)

Eigenvalues for A Singlet (S) and A Triplet (T)


,E(S)1,E(S)2,E(S)3,E(T)
0,-2.253906,-0.633991,-1.285266,-1.647682



Eigenvectors of A Singlet
 0.993643 0.0 0.112578
 0.000000 1.0 0.000000
-0.112578 0.0 0.993643

Matrix of A Singlet
-2.233375  0.181208  0.000000
 0.181208 -0.654521  0.000000
 0.000000  0.000000 -1.285266


In [6]:
#from change_all2 import *
from change_all_240523 import *




L,l,S,S_i,S_i2,U,t,t_i,t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = dist1_0A_bond0_740A()



#bond0_740A = [dist1_0A_bond0_740A(), dist1_2A_bond0_740A(), dist1_4A_bond0_740A(), dist1_6A_bond0_740A(), dist1_8A_bond0_740A(), dist2_0A_bond0_740A(), dist2_5A_bond0_740A(), dist3_0A_bond0_740A(),
#              dist3_5A_bond0_740A(), dist5_0A_bond0_740A(), dist7_5A_bond0_740A(), dist10_0A_bond0_740A(), dist20_0A_bond0_740A()]#, dist2_4A_bond0_740A(),dist2_6A_bond0_740A(),dist2_7A_bond0_740A()]
#bond0_740A = [dist1_8A_bond0_740A(), dist2_0A_bond0_740A(), dist2_4A_bond0_740A(),dist2_5A_bond0_740A(), dist2_6A_bond0_740A(),dist2_7A_bond0_740A(),dist3_0A_bond0_740A(),
#              dist3_5A_bond0_740A(), dist5_0A_bond0_740A(), dist10_0A_bond0_740A(), dist20_0A_bond0_740A()]

bond0_740A = [dist1_8A_bond0_740A(), dist1_9A_bond0_740A(), dist2_0A_bond0_740A(), dist2_1A_bond0_740A(), dist2_2A_bond0_740A(), dist2_3A_bond0_740A(), 
              dist2_4A_bond0_740A(), dist2_5A_bond0_740A(), dist2_6A_bond0_740A(),dist2_7A_bond0_740A(), dist2_8A_bond0_740A(), dist2_9A_bond0_740A(), 
              dist3_0A_bond0_740A(), dist3_5A_bond0_740A(), dist5_0A_bond0_740A(), dist10_0A_bond0_740A(), dist20_0A_bond0_740A()]


#bond2_220A = [dist1_0A_bond2_220A(), dist1_2A_bond2_220A(), dist1_4A_bond2_220A(), dist1_6A_bond2_220A(), dist1_8A_bond2_220A(), dist2_0A_bond2_220A(), dist2_5A_bond2_220A(), dist3_0A_bond2_220A(),
              #dist3_5A_bond2_220A(), dist5_0A_bond2_220A(), dist7_5A_bond2_220A(), dist10_0A_bond2_220A(), dist20_0A_bond2_220A()]


datas = pd.DataFrame(columns= ["E(S)1","E(S)2", "E(S)3", "E(T)"])

eigen = pd.DataFrame()
eigenbis = pd.DataFrame()

coeff_gs = 0
coeff_es = 1
coeff_os = 0

norm = coeff_gs ** 2 + coeff_es ** 2 + coeff_os ** 2

E1 = coeff_gs * numpy.sqrt(1 / norm)
E2 = coeff_es * numpy.sqrt(1 / norm)
E3 = coeff_os * numpy.sqrt(1 / norm)
print(E1,E2,E3)


for val in bond0_740A:
#for val in bond0_814A:
#for val in bond0_925A:
#for val in bond2_220A:
    
    L,l,S,S_i,S_i2,U,t,t_i,t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = val
    S_i = S_i2 = 0
    S = 0
    
    H, result_matrix, eigenvectors = Matrix_diag_b_SINGLET(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)
    datas = datas.append(result_matrix)
    eigen_frame = pd.DataFrame(eigenvectors)
    eigen = eigen.append(eigen_frame)


display(datas)
display(eigen)
#writer = ExcelWriter('output.xlsx')
#datas.to_excel('test_python_xls',engine='odf')

0.0 1.0 0.0


,E(S)1,E(S)2,E(S)3,E(T)
0,-1.627589,1.455028,-0.579156,-0.783767
0,-1.600810,1.429143,-0.578709,-0.783320
0,-1.576696,1.405710,-0.578368,-0.782979
0,-1.554885,1.384411,-0.578113,-0.782724
0,-1.535072,1.364975,-0.577924,-0.782535
0,-1.516998,1.347176,-0.577787,-0.782398
0,-1.500445,1.330820,-0.577688,-0.782299
0,-1.485227,1.315743,-0.577618,-0.782229
0,-1.471187,1.301802,-0.577568,-0.782179
0,-1.458193,1.288877,-0.577533,-0.782144


,0,1,2
0,0.999449,0.0,0.033206
1,0.000000,1.0,0.000000
2,-0.033206,0.0,0.999449
0,0.999429,0.0,0.033784
1,0.000000,1.0,0.000000
2,-0.033784,0.0,0.999429
0,0.999411,0.0,0.034323
1,0.000000,1.0,0.000000
2,-0.034323,0.0,0.999411
0,0.999393,0.0,0.034827


In [66]:
def Matrix_diag_b_TRIPLET(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):
    """
    Scheme :
        b|     |b'
         |     |
        a|     |a'
    
    This function diagonalizes one 3x3 matrix (4e- in 4MOs):    
        - H: molecule A is SINGLET and molecule B is a linear combination of SINGLET states

    Linear combination (normalized):
        B_state = coeff_gs * B_groundstate + coeff_es * B_excitedstate + coeff_os * B_openshell
    
    """
    # Initialization of results
    result = []
    #result2 = []
    
    # Columns for the DataFrame
    col = ("E(S)1","E(S)2", "E(S)3", "E(T)")
    
    # DEFINE THE TWO MATRIXES    
    H, singlet, quintet = Mat_aSinglet_bTriplet(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)
        
    # SCIPY METHOD of DIAGONALIZATION
    # https://coulomb.umontpellier.fr/perso/daniele.coslovich/python_for_physicists/#sec-3-3
    #eigenvalues2, eigenvectors2 = eigh(Hbis)
    # Obtain the eigenvalues and eigenvectors of H
    eigenvalues, eigenvectors = eigh(H)
    # Extract eigenvalues as real quantities
    eigenvalues = numpy.real(eigenvalues)
    #eigenvalues2 = numpy.real(eigenvalues2)

    # Join the results as lists of lists (here it is done only once but I kept the loop)
    #result = result + [[val for val in eigenvalues]]
    
    #result2 = result2 + [[val for val in eigenvalues2]]
    # Create the DataFrame with results and the columns
    
    
    #Data treatment
    new_eigenvectors =[[],[],[],[]]
    new_eigenvalues = [0,0,0,0]
    for i in range(4):
        if eigenvectors[i][0]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[0] = eigenvectors[i]
            new_eigenvalues[0] = eigenvalues[i]
            
        if eigenvectors[i][1]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[1] = eigenvectors[i]
            new_eigenvalues[1] = eigenvalues[i]
            
        if eigenvectors[i][2]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[2] = eigenvectors[i]
            new_eigenvalues[2] = eigenvalues[i]
            
        if eigenvectors[i][3]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[3] = eigenvectors[i]
            new_eigenvalues[3] = eigenvalues[i]
        
    result = [[val for val in new_eigenvalues]]
    df = pd.DataFrame(result, columns=col)  
    #df2 = pd.DataFrame(result2)
    df["E(S)"] = singlet
    df["E(Q)"] = quintet
    
    #print(result)
    
    return H, df, new_eigenvectors#,tripms1,ctripms1,cbiscbis

#from change_distance import *

# Import the integrals
L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = dist1_8A_bond0_740A()

#S_norm = 0.6598731215439126
#S = S_norm
#S = S_i = S_i2 = 0


#Kbis = omega2 = omega3 = theta1= theta2 = gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma6 = 0




H, result_matrix, eigenvectors = Matrix_diag_b_TRIPLET(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)

print("\033[4mEigenvalues for A Singlet (S) and A Triplet (T)\033[0m")
display(result_matrix)
print("\n\033[4mEigenvectors of A Singlet\033[0m")
#print(pd.DataFrame(eigenvectors).to_string(index=False, header=False))
display(pd.DataFrame(eigenvectors))
print("\n\033[4mMatrix of A Singlet\033[0m")
display(pd.DataFrame(H))#.to_string(index=False, header=False))
#print(pd.DataFrame(H).to_string(index=False, header=False))

#print(tripms1, ctripms1)
#print(cbiscbis)

#display(eigenvectorsbis)

Eigenvalues for A Singlet (S) and A Triplet (T)


,E(S)1,E(S)2,E(S)3,E(T),E(S),E(Q)
0,-1.528416,-0.006002,-0.613409,-1.06766,-1.276384,-0.890296



Eigenvectors of A Singlet


,0,1,2,3
0,0.993026,0.000000,0.000000,0.117896
1,0.000000,0.993299,-0.115571,0.000000
2,0.000000,0.115571,0.993299,0.000000
3,-0.117896,0.000000,0.000000,0.993026



Matrix of A Singlet


,0,1,2,3
0,-1.507256,0.178234,0.000000,0.000000
1,0.178234,-0.027163,0.000000,0.000000
2,0.000000,0.000000,-0.619476,-0.052147
3,0.000000,0.000000,-0.052147,-1.061593


In [104]:
# TRIPLET PART

#from change_all2 import *

from change_all_240523 import *

L,l,S,S_i,S_i2,U,t,t_i,t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = dist1_0A_bond0_740A()



#bond0_740A = [dist1_0A_bond0_740A(), dist1_2A_bond0_740A(), dist1_4A_bond0_740A(), dist1_6A_bond0_740A(), dist1_8A_bond0_740A(), dist2_0A_bond0_740A(), dist2_5A_bond0_740A(), dist3_0A_bond0_740A(),
#              dist3_5A_bond0_740A(), dist5_0A_bond0_740A(), dist7_5A_bond0_740A(), dist10_0A_bond0_740A(), dist20_0A_bond0_740A()]

#bond0_740A = [dist1_8A_bond0_740A(), dist2_0A_bond0_740A(), dist2_4A_bond0_740A(),dist2_5A_bond0_740A(), dist2_6A_bond0_740A(),dist2_7A_bond0_740A(),dist3_0A_bond0_740A(),
#              dist3_5A_bond0_740A(), dist5_0A_bond0_740A(), dist10_0A_bond0_740A(), dist20_0A_bond0_740A()]

bond0_740A = [dist1_8A_bond0_740A(), dist1_9A_bond0_740A(), dist2_0A_bond0_740A(), dist2_1A_bond0_740A(), dist2_2A_bond0_740A(), dist2_3A_bond0_740A(), 
              dist2_4A_bond0_740A(), dist2_5A_bond0_740A(), dist2_6A_bond0_740A(),dist2_7A_bond0_740A(), dist2_8A_bond0_740A(), dist2_9A_bond0_740A(), 
              dist3_0A_bond0_740A(), dist3_5A_bond0_740A(), dist5_0A_bond0_740A(), dist10_0A_bond0_740A(), dist20_0A_bond0_740A()]


datas = pd.DataFrame(columns= ["E(S)1","E(S)2", "E(S)3", "E(T)", "E(S)","E(Q)"])

eigen = pd.DataFrame()
eigenbis = pd.DataFrame()






for val in bond0_740A:
#for val in bond0_814A:
#for val in bond0_925A:
#for val in bond1_480A:
    
    L,l,S,S_i,S_i2,U,t,t_i,t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = val
    S = S_i = S_i2 = 0 
    #Kbis = omega2 = omega3 = theta1= theta2 = gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma6 = 0
    
   
    H, result_matrix, eigenvectors = Matrix_diag_b_TRIPLET(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)
    datas = datas.append(result_matrix)
    eigen_frame = pd.DataFrame(eigenvectors)
    eigen = eigen.append(eigen_frame)
    #print((t_i/S_i, t_i2/S_i2))

display(datas)
display(eigen)


,E(S)1,E(S)2,E(S)3,E(T),E(S),E(Q)
0,-2.900181,-0.778822,-2.327389,-2.518789,-2.757056,-2.560174
0,-2.884904,-0.778340,-2.321949,-2.514952,-2.744856,-2.545814
0,-2.873738,-0.777973,-2.317446,-2.512688,-2.735913,-2.535282
0,-2.865641,-0.777699,-2.313906,-2.511333,-2.729410,-2.527619
0,-2.859818,-0.777497,-2.311219,-2.510510,-2.724720,-2.522092
0,-2.855667,-0.777350,-2.309232,-2.510002,-2.721369,-2.518139
0,-2.852733,-0.777244,-2.307792,-2.509685,-2.718994,-2.515337
0,-2.850679,-0.777169,-2.306765,-2.509485,-2.717328,-2.513370
0,-2.849253,-0.777116,-2.306044,-2.509357,-2.716168,-2.512000
0,-2.848272,-0.777079,-2.305544,-2.509275,-2.715369,-2.511056


,0,1,2,3
0,0.998834,0.000000,0.000000,0.048283
1,0.000000,0.986884,0.161430,0.000000
2,0.000000,-0.161430,0.986884,0.000000
3,-0.048283,0.000000,0.000000,0.998834
0,0.998817,0.000000,0.000000,0.048623
...,...,...,...,...
3,-0.049501,0.000000,0.000000,0.998774
0,0.998774,0.000000,0.000000,0.049501
1,0.000000,1.000000,0.000000,0.000000
2,0.000000,0.000000,1.000000,0.000000
